<a href="https://colab.research.google.com/github/danishgufran/data_augmentation_ILS/blob/Dev/Code/DataAugmentationFunctions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
''' Author - Danish Gufran
           - Jadon Johnson
           - Isaac Blair
           - Sudeep Pasricha

EPIC Lab : Colorado State University, Fort Collins

<Danish.Gufran@colostate.edu>
<Jadon.Johnson@colostate.edu>
<Isaac.Blair@colostate.edu>
<Sudeep@colostate.edu>

Title - Data Augmentation Methods for Wi-Fi RSS Fingerprinting Based Indoor Localization.

Citation :

'''

' Author - Danish Gufran\n           - Jadon Johnson\n           - Isaac Blair\n           - Sudeep Pasricha\n\nEPIC Lab : Colorado State University, Fort Collins\n\n<Danish.Gufran@colostate.edu>\n<Jadon.Johnson@colostate.edu>\n<Isaac.Blair@colostate.edu>\n<Sudeep@colostate.edu>\n\nTitle - Data Augmentation Methods for Wi-Fi RSS Fingerprinting Based Indoor Localization.\n\nCitation :\n\n'

In [2]:
# !rm -rf maril
try:
  !git clone https://github.com/danishgufran/RSS_Database.git
  !git clone https://github.com/danishgufran/EPIC_Lab_Data.git
  !git clone https://github.com/EPIC-CSU/heterogeneous-rssi-indoor-nav.git
  !pip install tensorflow-addons
  !pip install keras-multi-head
  !pip install catboost
  !pip install cleverhans

except:
  from git import Repo  # pip install gitpython
  Repo.clone_from("https://github.com/danishgufran/RSS_Database.git")
  Repo.clone_from("https://github.com/danishgufran/EPIC_Lab_Data.git")
  Repo.clone_from("https://github.com/EPIC-CSU/heterogeneous-rssi-indoor-nav.git")

Cloning into 'RSS_Database'...
remote: Enumerating objects: 1521, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1521 (delta 0), reused 5 (delta 0), pack-reused 1516 (from 1)
Receiving objects: 100% (1521/1521), 182.30 MiB | 19.31 MiB/s, done.
Resolving deltas: 100% (738/738), done.
Updating files: 100% (1655/1655), done.
Cloning into 'EPIC_Lab_Data'...
remote: Enumerating objects: 328, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 328 (delta 79), reused 105 (delta 77), pack-reused 221 (from 1)
Receiving objects: 100% (328/328), 86.35 MiB | 23.70 MiB/s, done.
Resolving deltas: 100% (162/162), done.
Cloning into 'heterogeneous-rssi-indoor-nav'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 80 (delta 1), reused 0 (delta 0), pack-reused 74 (from 1)
Receiving

In [11]:
import numpy as np
from google.colab import files
import shutil

import io
from contextlib import redirect_stdout
from zipfile import ZipFile

import copy
from copy import deepcopy
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D , LSTM, Attention
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import*
import random as random
import time
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd

import RSS_Database.Stone_Seth.Seth
from RSS_Database.Stone_Seth.Seth import fetch_seth, Devices, Floorplan, get_mac_ids

from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from EPIC_Lab_Data.data import Devices, Floorplan, build_dataset
from EPIC_Lab_Data.helpers import compute_distances
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from numpy import loadtxt
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#import lightgbm as lgb

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import numpy as np

import logging
import json
import numpy as np
import os
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import LayerNormalization
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.losses import Loss
from tensorflow.keras.optimizers import Adam
from sklearn import svm
import xgboost as xgb

from EPIC_Lab_Data.helpers import split_frame, compute_distances
from EPIC_Lab_Data.data import build_dataset
from EPIC_Lab_Data.Maril.MultiHeadAttentionAddon import MultiHeadAttentionAddon

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.momentum_iterative_method import momentum_iterative_method
from cleverhans.tf2.attacks.carlini_wagner_l2 import carlini_wagner_l2
from cleverhans.tf2.attacks.basic_iterative_method import basic_iterative_method

In [8]:
def train_data(itr,dev, floorplan):
    # dfs is a list of dataframes
# meta is a dataframe with meta data

#getting train data

#     train_fp, train_meta = fetch_seth(
#     dev,
#     str(floorplan),
#     ci = int(itr),
#     base_path="RSS_DataEPIC_Lab_Data/Data/train/"  # <-- this would be 'seth/temp/clean' from outside this dir
# )
    train_fp, _, macs, lbl2cord = build_dataset(
        dev,
        str(floorplan),
    )
    train_fp = train_fp.sample(frac=1).reset_index(drop=True)
    train_aps = get_mac_ids(train_fp.columns)
    train_x = train_fp[train_aps].values
    #train_x = (train_x + 100)/100
    train_y = (train_fp["label"]).values
    return train_x, train_y, train_aps

def test_data(itr, train_aps, dev, floorplan):
    #getting test data
#     test_fp, test_meta = fetch_seth(
#     dev ,
#     str(floorplan),
#     ci = itr,
#     base_path="RSS_DataEPIC_Lab_Data/Data/test/"  # <-- this would be 'seth/temp/clean' from outside this dir
# )
    train_df, test_fp, macs_test, lbl2cords = build_dataset(
          dev,
          str(floorplan)
      )
    test_aps = get_mac_ids(test_fp.columns)
    missing_aps = list(set(train_aps)-set(test_aps))
    missing_df = pd.DataFrame(0, index=test_fp.index, columns=missing_aps)
    test_fp = pd.concat([test_fp, missing_df], axis=1)
    test_x = test_fp[train_aps].values
    #test_x = (test_x + 100)/100
    test_y = (test_fp["label"]).values

    # test_x = (test_x + 100)/100


    return np.array(test_x), test_y

def temp_train_data(dev, floorplan, ci_val):
    # dfs is a list of dataframes
# meta is a dataframe with meta data

#getting train data

    train_fp, train_meta = fetch_seth(
    dev,
    str(floorplan),
    ci = ci_val,
    base_path="RSS_Database/Stone_Seth/temp/clean/"  # <-- this would be 'seth/temp/clean' from outside this dir
)
    # train_fp, _, macs, lbl2cord = build_dataset(
    #     dev,
    #     str(floorplan),
    # )
    train_fp = train_fp.sample(frac=1).reset_index(drop=True)
    train_aps = get_mac_ids(train_fp.columns)
    train_x = train_fp[train_aps].values
    train_x = (train_x + 100)/100
    train_y = (train_fp["label"]).values
    return train_x, train_y, train_aps
def temp_test_data(train_aps, dev, floorplan, ci_val):
    #getting test data
    test_fp, test_meta = fetch_seth(
    str(dev) ,
    str(floorplan),
    ci = ci_val,
    base_path="RSS_Database/Stone_Seth/temp/clean/"  # <-- this would be 'seth/temp/clean' from outside this dir
)
    # train_df, test_fp, macs_test, lbl2cords = build_dataset(
    #       dev,
    #       str(floorplan)
    #   )
    test_aps = get_mac_ids(test_fp.columns)
    missing_aps = list(set(train_aps)-set(test_aps))
    test_fp[missing_aps] = 0
    test_x = test_fp[train_aps].values
    test_x = (test_x + 100)/100
    test_y = (test_fp["label"]).values
    return test_x, test_y

In [9]:
def stacked_sae(input, train_x, train_y, eph):
  print("\n Training Stacked SAE... \n")
  h1 = int(input - input*0.3)
  h2 = int(h1 - h1*0.3)
  h3 = int(h2 - h2*0.3)
  h4 = int(h3 - h3*0.3)
  #print(input,h1,h2,h3, h4)
  # AE -1
  input_img = keras.layers.Input(shape = (input, ))
  GN = keras.layers.GaussianNoise(0.)(input_img)
  distorted_input1 = Dropout(.1)(GN)
  encoded1 = Dense(h1, activation = 'gelu')(distorted_input1)
  decoded1 = Dense(input, activation = 'sigmoid')(encoded1)

  autoencoder1 = keras.models.Model(inputs = input_img, outputs = decoded1)
  encoder1 = keras.models.Model(inputs = input_img, outputs = encoded1)

  # AE -2
  encoded1_input = keras.layers.Input(shape = (h1,))
  GN1 = keras.layers.GaussianNoise(0.)(encoded1_input)
  distorted_input2 = Dropout(.10)(GN1)
  encoded2 = Dense(h2, activation = 'gelu')(distorted_input2)
  decoded2 = Dense(h1, activation = 'sigmoid')(encoded2)

  autoencoder2 = tf.keras.Model(inputs = encoded1_input, outputs = decoded2)
  encoder2 = tf.keras.Model(inputs = encoded1_input, outputs = encoded2)

  # AE -3
  encoded2_input = keras.layers.Input(shape = (h2,))
  GN2 = keras.layers.GaussianNoise(0.)(encoded2_input)
  distorted_input3 = Dropout(.10)(GN2)
  encoded3 = Dense(h3, activation = 'gelu')(distorted_input3)
  decoded3 = Dense(h2, activation = 'sigmoid')(encoded3)

  autoencoder3 = tf.keras.Model(inputs = encoded2_input, outputs = decoded3)
  encoder3 = tf.keras.Model(inputs = encoded2_input, outputs = encoded3)

  # AE -4
  encoded3_input = keras.layers.Input(shape = (h3,))
  GN3 = keras.layers.GaussianNoise(0.)(encoded3_input)
  distorted_input4 = Dropout(.10)(GN3)
  encoded4 = Dense(h4, activation = 'gelu')(distorted_input4)
  decoded4 = Dense(h3, activation = 'sigmoid')(encoded4)

  autoencoder4 = tf.keras.Model(inputs = encoded3_input, outputs = decoded4)
  encoder4 = tf.keras.Model(inputs = encoded3_input, outputs = encoded4)
  # Final AE
  encoded1_da = Dense(h1, activation = 'sigmoid')(input_img)
  encoded2_da = Dense(h2, activation = 'sigmoid')(encoded1_da)
  encoded3_da = Dense(h3, activation = 'sigmoid')(encoded2_da)
  encoded4_da = Dense(h4, activation = 'sigmoid')(encoded3_da)
  decoded4_da = Dense(h3, activation = 'sigmoid')(encoded4_da)
  decoded3_da = Dense(h2, activation = 'sigmoid')(decoded4_da)
  decoded2_da = Dense(h1, activation = 'sigmoid')(decoded3_da)
  decoded1_da = Dense(input, activation = 'sigmoid')(decoded2_da)

  deep_autoencoder = tf.keras.Model(inputs = input_img, outputs = decoded1_da)
  #compile
  sgd1 = tf.keras.optimizers.Adam(learning_rate = 0.001)
  sgd2 = tf.keras.optimizers.Adam(learning_rate = 0.001)
  sgd3 = tf.keras.optimizers.Adam(learning_rate = 0.001)
  sgd4 = tf.keras.optimizers.Adam(learning_rate = 0.001)

  sgdD = tf.keras.optimizers.Adam(learning_rate = 0.001)

  autoencoder1.compile(loss='mse', optimizer = sgd1)
  autoencoder2.compile(loss='mse', optimizer = sgd2)
  autoencoder3.compile(loss='mse', optimizer = sgd3)
  autoencoder4.compile(loss='mse', optimizer = sgd4)

  encoder1.compile(loss='mse', optimizer = sgd1)
  encoder2.compile(loss='mse', optimizer = sgd1)
  encoder3.compile(loss='mse', optimizer = sgd1)
  encoder4.compile(loss='mse', optimizer = sgd1)

  deep_autoencoder.compile(loss='mse', optimizer = sgdD)
  # fit ae 1
  autoencoder1.fit(train_x, train_x,
                epochs = eph,
                validation_split = 0.0020,
                shuffle = False, verbose =0,
                callbacks=[
                keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)])
  first_layer_code = encoder1.predict(train_x)
  #fit ae 2
  autoencoder2.fit(first_layer_code, first_layer_code,
                epochs = eph,
                validation_split = 0.0020,
                shuffle = False, verbose =0,
                callbacks=[
                keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)])
  second_layer_code = encoder2.predict(first_layer_code)
  #fit ae 3
  autoencoder3.fit(second_layer_code, second_layer_code,
               epochs = eph,
               validation_split = 0.0020,
               shuffle = False, verbose =0,
                callbacks=[
                keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)])
  third_layer_code = encoder3.predict(second_layer_code)
    #fit ae 4
  autoencoder4.fit(third_layer_code, third_layer_code,
               epochs = eph,
               validation_split = 0.0020,
               shuffle = False, verbose =0,
                callbacks=[
                keras.callbacks.EarlyStopping(patience=2000, restore_best_weights=True)])
  fourth_layer_code = encoder4.predict(third_layer_code)
  # Setting the weights of the deep autoencoder
  deep_autoencoder.layers[1].set_weights(encoder1.layers[-1].get_weights()) # first dense layer

  deep_autoencoder.layers[2].set_weights(encoder2.layers[-1].get_weights()) # second dense layer

  deep_autoencoder.layers[3].set_weights(encoder3.layers[-1].get_weights()) # thrird dense layer

  deep_autoencoder.layers[4].set_weights(encoder4.layers[-1].get_weights()) # fourth dense layer

  # deep_autoencoder.layers[5].set_weights(autoencoder4.layers[-1].get_weights()) # fourth dense layer

  deep_autoencoder.layers[6].set_weights(autoencoder3.layers[-1].get_weights()) # first decoder
  deep_autoencoder.layers[7].set_weights(autoencoder2.layers[-1].get_weights()) # second decoder
  deep_autoencoder.layers[8].set_weights(autoencoder1.layers[-1].get_weights()) # third decoder

  deep_autoencoder.fit(train_x, train_x,
                epochs = eph,
                validation_split = 0.0020,
                shuffle = False,
                verbose =0,
                callbacks=[keras.callbacks.EarlyStopping(patience=500, restore_best_weights=True)])

  sae_fingerprint = []

  for fingerprint in train_x:
    fingerprint = fingerprint.reshape(1, -1)
    sae_train_x = deep_autoencoder.predict(fingerprint, verbose = 0)
    sae_train_x = np.array(sae_train_x)
    rounded_data = np.round(sae_train_x[0], 2)  # Round to 2 decimal places
    sae_fingerprint.append(rounded_data)

  return np.array(sae_fingerprint), train_y, deep_autoencoder


In [12]:
def sae(input_dim, train_x, train_y, epochs):
    print("\n Training Standard Autoencoder... \n")

    # Define the architecture
    h1 = int(input_dim - input_dim * 0.3)
    h2 = int(h1 - h1 * 0.3)
    h3 = int(h2 - h2 * 0.3)
    h4 = int(h3 - h3 * 0.3)

    # Input layer
    input_img = keras.layers.Input(shape=(input_dim,))

    # Encoder
    encoded = Dense(h1, activation='gelu')(input_img)
    encoded = Dense(h2, activation='gelu')(encoded)
    encoded = Dense(h3, activation='gelu')(encoded)
    encoded = Dense(h4, activation='gelu')(encoded)

    # Decoder
    decoded = Dense(h3, activation='gelu')(encoded)
    decoded = Dense(h2, activation='gelu')(decoded)
    decoded = Dense(h1, activation='gelu')(decoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)

    # Autoencoder model
    autoencoder = keras.models.Model(inputs=input_img, outputs=decoded)

    # Compile the model
    autoencoder.compile(optimizer='adam', loss='mse')

    # Train the autoencoder
    autoencoder.fit(train_x, train_x,
                    epochs=epochs,
                    validation_split=0.0020,
                    shuffle=True,
                    verbose=0,
                    callbacks=[keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)])

    # Encode the input data to generate the final feature representation
    encoded_data = autoencoder.predict(train_x)

    # Optionally round the encoded data
    sae_fingerprint = np.round(encoded_data, 2)

    return sae_fingerprint, train_y, autoencoder

In [13]:

# Generate augmented data using GAN

def build_generator(noise_dim, output_dim):
    model = models.Sequential()
    model.add(layers.Dense(noise_dim, input_dim=noise_dim, activation='relu'))
    # model.add(layers.Dense(64, activation='tanh'))
    model.add(layers.Dense(int(noise_dim), activation='relu'))
    model.add(layers.Dense(output_dim, activation='sigmoid'))
    return model

# Discriminator Network
def build_discriminator(input_dim):
    model = models.Sequential()
    model.add(layers.Dense(128, input_dim=input_dim, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    # model.add(layers.Dense(32, activation='tanh'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

def train_gan(train_x, train_y, noise_dim, batch_size, epochs):
    input_dim = train_x.shape[1]
    generator = build_generator(noise_dim, input_dim)
    discriminator = build_discriminator(input_dim)

    # Compile Discriminator
    discriminator.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Combined model (Generator + Discriminator)
    discriminator.trainable = False
    gan_input = layers.Input(shape=(noise_dim,))
    generated_data = generator(gan_input)
    gan_output = discriminator(generated_data)
    gan = models.Model(gan_input, gan_output)
    gan.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy')

    # Training the GAN
    print("\n Training GAN... \n")
    for epoch in range(epochs):
        for _ in range(len(train_x) // batch_size):
            # Generate fake data
            noise = np.random.normal(0, 1, (batch_size, noise_dim))
            generated_data = generator.predict(noise, verbose=0)

            # Select a random batch of real data
            idx = np.random.randint(0, train_x.shape[0], batch_size)
            real_data = train_x[idx]

            # Labels for real and fake data
            real_labels = np.ones((batch_size, 1))
            fake_labels = np.zeros((batch_size, 1))

            # Train the Discriminator
            d_loss_real = discriminator.train_on_batch(real_data, real_labels)
            d_loss_fake = discriminator.train_on_batch(generated_data, fake_labels)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train the Generator
            noise = np.random.normal(0, 1, (batch_size, noise_dim))
            valid_y = np.ones((batch_size, 1))
            g_loss = gan.train_on_batch(noise, valid_y)

        # Uncomment below for training info (verbose = 1)
        # if epoch :
        #     print(f'Epoch {epoch}, D Loss: {d_loss[0]:.4f}, D Acc.: {100*d_loss[1]:.2f}%')

    # Generate augmented data
    gan_x = []
    gan_y = []
    for i in range(len(train_x)):
        noise = np.random.normal(0, 1, (1, noise_dim))
        generated_data = generator.predict(noise, verbose=0)
        rounded_data = np.round(generated_data[0], 2)  # Round to 2 decimal places
        gan_x.append(rounded_data)
        gan_y.append(train_y[i])  # Assuming RP class remains the same

    sorted_indices = np.argsort(train_y)
    gan_x = np.array(gan_x)[sorted_indices]
    gan_y = np.array(gan_y)[sorted_indices]
    return gan_x, gan_y, generator


In [14]:

def train_fgsm(train_x, train_y, epsilon, model, aug, model_name, train, flp):
    # Convert train_x and train_y to TensorFlow tensors
    train_x = tf.convert_to_tensor(train_x, dtype=tf.float32)
    train_y = tf.convert_to_tensor(train_y, dtype=tf.float32)

    # Ensure model is compiled if not already
    try:
      if not model.compiled_loss:
        model.compile(optimizer='adam', loss='categorical_crossentropy')
    except:
    #add safety DNN here
      model = load_model("BackupDNN.keras")
      model.compile(optimizer='adam', loss='categorical_crossentropy')

    # Generate FGSM adversarial samples
    fgsm_x = fast_gradient_method(model_fn=model, x=train_x, eps=epsilon, norm=np.inf)

    return np.array(fgsm_x), train_y


def train_pgd(train_x, train_y, epsilon, model, alpha=0.01, num_iterations=40):
    # Convert train_x and train_y to TensorFlow tensors
    train_x = tf.convert_to_tensor(train_x, dtype=tf.float32)
    #train_y = tf.convert_to_tensor(train_x, dtype=tf.float32) <-- this is not needed and messes with dimensions of output

    # Ensure model is compiled if not already
    try:
      if not model.compiled_loss:
        model.compile(optimizer='adam', loss='categorical_crossentropy')
    except:
    #add safety DNN here
      model = load_model("BackupDNN.keras")
      model.compile(optimizer='adam', loss='categorical_crossentropy')

    # Generate PGD adversarial samples
    pgd_x = projected_gradient_descent(model_fn=model, x=train_x, eps=epsilon, eps_iter=alpha, nb_iter=num_iterations, norm=np.inf)

    return np.array(pgd_x), train_y


def train_mim(train_x, train_y, epsilon, model, alpha=0.01, num_iterations=40):
    # Convert train_x and train_y to TensorFlow tensors
    train_x = tf.convert_to_tensor(train_x, dtype=tf.float32)
    #train_y = tf.convert_to_tensor(train_y, dtype=tf.float32)

    # Ensure model is compiled if not already
    try:
      if not model.compiled_loss:
        model.compile(optimizer='adam', loss='categorical_crossentropy')
    except:
    #add safety DNN here
      model = load_model("BackupDNN.keras")
      model.compile(optimizer='adam', loss='categorical_crossentropy')

    # Generate MIM adversarial samples
    mim_x = momentum_iterative_method(model_fn=model, x=train_x, eps=epsilon, eps_iter=alpha, nb_iter=num_iterations, norm=np.inf)

    return np.array(mim_x), train_y

def train_bim(train_x, train_y, epsilon, model, alpha=0.01, num_iterations=40):
  # Convert train_x and train_y to TensorFlow tensors, this causes issues with catboost
  train_x = tf.convert_to_tensor(train_x, dtype=tf.float32)
  #train_y = tf.convert_to_tensor(train_y, dtype=tf.float32)

  # Ensure model is compiled if not already
  try:
    if not model.compiled_loss:
      model.compile(optimizer='adam', loss='categorical_crossentropy')
  except:
  #add safety DNN here
    model = load_model("BackupDNN.keras")
    model.compile(optimizer='adam', loss='categorical_crossentropy')
  # Generate BIM adversarial samples
  bim_x = basic_iterative_method(model_fn=model, x=train_x, eps=epsilon, eps_iter=alpha, nb_iter=num_iterations, norm=np.inf)

  return np.array(bim_x), train_y

def train_cw(train_x, train_y, epsilon, model):
  # Convert train_x and train_y to TensorFlow tensors
  train_x = tf.convert_to_tensor(train_x, dtype=tf.float32)
  #train_y = tf.convert_to_tensor(train_y, dtype=tf.float32)

  # Ensure model is compiled if not already
  try:
    if not model.compiled_loss:
      model.compile(optimizer='adam', loss='categorical_crossentropy')
  except:
  #add safety DNN here
    model = load_model("BackupDNN.keras")
    model.compile(optimizer='adam', loss='categorical_crossentropy')
  # Generate BIM adversarial samples
  cw_x = carlini_wagner_l2(model_fn=model, x=train_x, clip_min = -100.0)

  return np.array(cw_x), train_y


In [15]:
def model_params(model, save_name):
  # The summary can be commented to not print on console [Will still save in .txt]
    print(f'\n*** Model Summary : {save_name} ***\n')
    if isinstance(model, CatBoostClassifier):
      params = model.get_all_params() #this is a build in CBC method, returns a dict
      with open(f'{save_name}.txt', 'w') as f:
        for key, value in params.items():
            f.write(f'{key}: {value}\n')
        f.close()
    else:
      model.summary()
      with open(f'{save_name}.txt', 'w') as f:
          model.summary(print_fn=lambda x: f.write(x + '\n'))
          f.close()

In [16]:
'''
Add functions for the models here :

# Add functions for :
# KNN, GPC, SVM, RF, CatBoost, XgBoost, LightGBM, MLP, DNN, CNN, Attention, Transformer, VIT

[1] Feel free to pass any arguments as input to the functions.
[2] Each function must always return the trained model #return model
[3] The DNN model is NOT optimized - you need to fix it
[4] Play with the hyper-parameters to get the best trained model (you can change things as you seem fit)
[5] You can use grid search methods like : NAS, Keras Tuner, Auto Keras and so on, to find the best hyper parameters
    Note : It would be best if you open a new cell to test out your grid search method on the ML model and once you find the best model configuration you can insert that to the main run.
'''



def train_DNN(train_x, train_y, ep = 50, batch_size=128):
    print('Training DNN Classifier \n')
    # Build the DNN model
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Dense(350, activation='gelu',input_shape=train_x.shape[1:]))

    model.add(tf.keras.layers.GaussianNoise(0.15))
    model.add(tf.keras.layers.Dropout(0.20))

    model.add(tf.keras.layers.Dense(192, activation='gelu', kernel_regularizer=keras.regularizers.l2(0.01)))

    model.add(tf.keras.layers.GaussianNoise(0.15))
    model.add(tf.keras.layers.Dropout(0.20))

    model.add(tf.keras.layers.Dense(128, activation='gelu', kernel_regularizer=keras.regularizers.l2(0.01)))

    model.add(tf.keras.layers.GaussianNoise(0.15))
    model.add(tf.keras.layers.Dropout(0.20))

    model.add(tf.keras.layers.Dense(192, activation='gelu', kernel_regularizer=keras.regularizers.l2(0.01)))

    model.add(tf.keras.layers.Dense(int(max(train_y)+50), activation='gelu'))

    model.add(tf.keras.layers.Dense(int(max(train_y)+1), activation='softmax'))

    # Compile the model
    model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

    # Train the model
    model.fit(train_x,train_y,
                epochs = ep,
                shuffle = True,
                verbose = 0)

    return model

def train_Catboost(train_x, train_y, iter):

  model = CatBoostClassifier(
      iterations = iter, # 1000 are ideal
      loss_function='MultiClass',
      bootstrap_type = "Bayesian",
      eval_metric = 'MultiClass',
      leaf_estimation_iterations = 100,
      random_strength = 0.5,
      depth = 7,
      l2_leaf_reg = 5,
      learning_rate=0.1,
      bagging_temperature = 0.5,
      task_type = "GPU",
      silent = True
  )
  model.fit(train_x, train_y)

  return model

def train_XGboost(train_x, train_y):
  model = XGBClassifier(
      learning_rate = 0.3,
      gamma	= 1,
      max_depth	= 6,
      colsample_bytree = 1,
      reg_lambda =1,
      n_estimators = 100,
      scale_pos_weight = 1,
      booster = "gbtree",
      #tree_method = "gpu_hist",
      subsample = 1
      )
  model.fit(train_x, train_y)
  return model

In [ ]:
''' Code Instructions :

[1] Create new functions (in the cell above) to train all ML models (I have added research papers for each model for you to reference)
        Please Note : The research papers are for reference to see how the particular model is trained. Look at the hyper-parameters the paper mentions and start with those, after which you can fine tune them.
        These are the model -> KNN, GPC, SVM, RF, CatBoost, XgBoost, LightGBM, MLP, DNN, CNN, Attention, Transformer, VIT
[2] Once the model functions are created call them in the cell below look out for '# [JJ] [IB] :' for additional information
[3] Next update the model_name variable with the name of the new model
[4] Use the new model name to add subsequent 'if' conditions for all training conditions
[5] Now you need to fine tune the hyper-parameters in your new model to get the best trained model.
    Here is how you can vaidate your model:
    [5.A] Monitor the the accuracy and loss functions during training (accuracy must increase and loss must decrease)
    [5.B] No need to change anything in SAE, Stacked SAE, GAN, FGSM, PGD, and MIM - Only update the new model functions
    [5.C] Make sure to add all import and API calls in the 3rd cell of this notebook
    [5.D] Make sure to add any pip installs (if needed) in the second cell of this notebook
    [5.E] Finally, check the summary (CSV) files that gets generated after inference to see the average error for each case, ideally we need lower errors for all or MOST cases
          One way to evaluate if your model is trained well is to see if it atleast (pay attention here) has lower errors for the conditions it was trained on
          E.g, If the model was trained only on ORIGINAL data it must show low errors for inference on original data (across all devices) : This is the bare minimum, if other inferences like FGSM and so on show low errors as well then you have a great model (You need to aim to get this)
          E.g, If the model was trained only on ORIGINAL + FGSM data it must show low errors for inference on original and FGSM data (across all devices)
    [5.F] At the end you will notice that the script downloads a .zip file with the trained model (.keras), model summary (.txt), and inference output (.csv) - also another .csv file with the summary (average errors across all inference conditions). You will report that .zip (assuming with the best hyper-parameters of the model) to me.
[6] Before you submit the .zip file and the updated .ipynb (with your additions to the ML model) make sure your team sits together and merge all changes into 1 .ipynb (so that i have the most recent notebook with changes from everyone)
    [6.A] During the merge make sure the script has no errors and the results are satisfactory.
    [6.B] Take this oppurtunity to evaluate each others changes to check if the results are satisfactory. Only report data to me when both of you are convinced with the results. I will do one last check before we move on to Phase 4 (As discussed in our meetings).

'''

In [ ]:
'''
Here are some research papers for you to get started with. Look at the hyper-parameters the paper proposes and start with those and then use other sources (internet) to learn the uses of different hyper-parameters and start experimenting on them.
You can also create a temporary cell somewhere below to test your model before you run it in the main cell.
The main cell may run for a couple of hours, so make sure there are no errors as you go.

Here are the list of research papers:

KNN [1], GPC [2], SVM [3], RF [4], CatBoost [5], XgBoost [6], LightGBM [7], MLP [8], DNN [9], CNN [10], Attention [11], Transformer [12], VIT [13]

[1] KNN : https://ieeexplore.ieee.org/abstract/document/7386596

[2] GPC : https://ieeexplore.ieee.org/abstract/document/8767421

[3] SVM : https://ieeexplore.ieee.org/abstract/document/7986446

[4] RF : https://link.springer.com/article/10.1007/s11277-020-07977-w

[5] CatBoost : https://ieeexplore.ieee.org/abstract/document/10130598

[6] XgBoost : https://www.mdpi.com/1424-8220/22/17/6629

[7] LightGBM : https://www.mdpi.com/1424-8220/21/8/2722

[8] MLP : https://ieeexplore.ieee.org/abstract/document/9606541

[9] DNN : https://dl.acm.org/doi/full/10.1145/3607919

[10] CNN : https://ieeexplore.ieee.org/abstract/document/9060340

[11] Attention : https://ieeexplore.ieee.org/abstract/document/9918120

[12] Transformer : https://ieeexplore.ieee.org/abstract/document/9923937

[13] VIT : https://ieeexplore.ieee.org/abstract/document/10247684

[13] VIT : https://arxiv.org/abs/2010.11929
'''

# NAS, Keras tuner, auto keras

In [ ]:
from keras.regularizers import l2

train_x, train_y, train_aps = train_data(0,"OP3", 'engr0')
train = 'OP3'
flp = 'engr0'
train_ap_dict = {}
train_ap_dict[f'{train}_{flp}'] = train_aps
sorted_indices = np.argsort(train_y)
train_x = train_x[sorted_indices]
train_y = train_y[sorted_indices]

#model = train_DNN(train_x, train_y, 300, batch_size=128)
print('Training DNN Classifier \n')
    # Build the DNN model
model = keras.Sequential([
    layers.Dense(350, activation='gelu', input_shape=train_x.shape[1:], kernel_regularizer=l2(0.001)),
    layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(center = True, scale = False),

    layers.Dense(128, activation='relu'),
    #layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(center = True, scale = False),

    layers.Dense(100, activation='relu'),
    #layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(center = True, scale = False),

    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    tf.keras.layers.BatchNormalization(center = True, scale = False),

    layers.Dense(int(max(train_y)+1), activation='sigmoid'),
])
#model.add(tf.keras.layers.Dense(350, activation='relu',input_shape=train_x.shape[1:]))
#model.add(tf.keras.layers.Dense(int(max(train_y)+1), activation='softmax'))

    # Compile the model
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate = 0.005),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

    # Train the model
history = model.fit(train_x,train_y, epochs = 600,shuffle = True, verbose = 0)
print(history.history['accuracy'])
print(history.history['loss'])



In [31]:
train_x, train_y, train_aps = train_data(0,"OP3", 'engr0')
train = 'OP3'
flp = 'engr0'
train_ap_dict = {}
train_ap_dict[f'{train}_{flp}'] = train_aps
sorted_indices = np.argsort(train_y)
#train_x = train_x[sorted_indices]
#train_y = train_y[sorted_indices]

model = train_Catboost(train_x, train_y, 50)

In [46]:
ci_val = 0
train_aps_value = train_ap_dict[f'{train}_{flp}']
test_x, test_y = test_data(ci_val, train_aps_value, train, flp)
predictions = model.predict(test_x)
pred_class = []
for prediction in predictions:
  pred_class.append(prediction[0])
pred_class = np.array(pred_class)

prediction_classes = np.argmax(predictions, axis=-1)

mean_error = np.abs(pred_class - test_y)
print("mean_error:")
print(mean_error)
print("prediction_classes:")
print(pred_class)

#print(predictions[0].shape)

print(pred_class.shape)
print(test_y.shape)



mean_error:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
prediction_classes:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60]
(61,)
(61,)


In [49]:
%%capture
# KNN, GPC, SVM, RF, CatBoost, XgBoost, LightGBM, MLP, DNN, CNN, Attention, Transformer, VIT

# [JJ] [IB] : Make sure to update this variable while running other models
# Define the ML model to train
model_name = 'DNN' #'CATBOOST' #'XGBOOST' #'DNN'

# Training Device
train = 'OP3'

# Testing Device
dev = ['BLU','HTC','LG','MOTO','OP3','S7', 'i12p', 'nk7', 'pxl4']

# Building Floorplans
# ['engr0', 'engr1', 'glover', 'sciences', 'libstudy']
floorplan = ['engr0', 'engr1', 'glover', 'sciences', 'libstudy']

# Different augmentations to train the ML model
augmented_data = ['ORIGINAL','SAE', 'STACKED_SAE', 'GAN', 'FGSM', 'PGD', 'MIM', 'BIM']

mode = ['ORIGINAL', 'SAE', 'STACKED_SAE', 'GAN']

# Test the trained model on different conditions
attack = ['ORIGINAL', 'FGSM', 'PGD', 'MIM', 'BIM']

# Perturbation strength for Adversarial Methods (FGSM, PGD, MIM)
# [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
epsilon = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

train_ap_dict = {}

for flp in floorplan:
    print(f'\n Train dev -> {train}   flp -> {flp} \n')
    # Initial CI only !!!
    ci_val = 0
    train_x, train_y, train_aps = train_data(ci_val,train, flp)
    train_ap_dict[f'{train}_{flp}'] = train_aps
    sorted_indices = np.argsort(train_y)
    train_x = train_x[sorted_indices]
    train_y = train_y[sorted_indices]

    for aug in augmented_data:
      if aug == 'ORIGINAL':
        print(f'Training on Original RSS - Train Device : {train}  Floorplan : {flp} \n')

        print(f'\n************ Training Model : {aug} {flp} + {model_name} ************\n')

        # [JJ] [IB] : Add more if conditions for other models
        if model_name == 'DNN':
          model = train_DNN(train_x, train_y, 700, batch_size=128)
          save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
          model.save(f'{save_name}.keras')
          model.save("BackupDNN.keras")
          model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

        print(f'Training ORIGINAL Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')

        if model_name == 'CATBOOST':
          model = train_Catboost(train_x, train_y, 50)
          save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
          model.save_model(f'{save_name}.cbm')
          model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

        print(f'Training ORIGINAL Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')

        if model_name == 'XGBOOST':
          model = train_XGboost(train_x, train_y)
          save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
          model.save_model(f'{save_name}.json')
          #model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

        print(f'Training ORIGINAL Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')

      if aug == 'SAE':
        print(f'Training on SAE  - Train Device : {train}  Floorplan : {flp} \n')
        # Replace this with other models to train
        sae_x, sae_y, sae_model = sae(int(train_x.shape[-1]), train_x, train_y, 350)
        model_params(sae_model, f'{aug}_{train}_{flp}')
        sae_org_x = np.concatenate((train_x, sae_x), axis=0)
        sae_org_y = np.concatenate((train_y, sae_y), axis=0)
        print(f'\n************ Training Model : {aug} {flp} + {model_name} ************\n')

        # [JJ] [IB] : Add more if conditions for other models
        if model_name == 'DNN':
          model = train_DNN(sae_org_x, sae_org_y, 700, batch_size=128)
          save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
          model.save(f'{save_name}.keras')
          # model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

        print(f'Training SAE Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')

        if model_name == 'CATBOOST':
          model = train_Catboost(sae_org_x, sae_org_y, 50)
          save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
          model.save_model(f'{save_name}.cbm')
          model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

        print(f'Training SAE Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')

        if model_name == 'XGBOOST':
          model = train_XGboost(sae_org_x, sae_org_y)
          save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
          model.save_model(f'{save_name}.json')
          #model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

        print(f'Training SAE Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')

      if aug == 'STACKED_SAE':
        print(f'Training on Stacked SAE - Train Device : {train}  Floorplan : {flp} \n')
        # Replace this with other models to train
        stacked_sae_x, stacked_sae_y, stacked_sae_model = stacked_sae(int(train_x.shape[-1]), train_x, train_y, 300)
        model_params(stacked_sae_model, f'{aug}_{train}_{flp}')
        sta_sae_org_x = np.concatenate((train_x, stacked_sae_x), axis=0)
        sta_sae_org_y = np.concatenate((train_y, stacked_sae_y), axis=0)
        print(f'\n************ Training Model : {aug} {flp} + {model_name} ************\n')

        # [JJ] [IB] : Add more if conditions for other models
        if model_name == 'DNN':
          model = train_DNN(sta_sae_org_x, sta_sae_org_y, 700, batch_size=128)
          save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
          model.save(f'{save_name}.keras')
          # model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

        print(f'Training STACKED SAE Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')

        if model_name == 'CATBOOST':
          model = train_Catboost(sta_sae_org_x, sta_sae_org_y, 50)
          save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
          model.save_model(f'{save_name}.cbm')
          model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

        print(f'Training STACKED SAE Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')

        if model_name == 'XGBOOST':
          model = train_XGboost(sta_sae_org_x, sta_sae_org_y)
          save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
          model.save_model(f'{save_name}.json')
          #model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

        print(f'Training STACKED SAE Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')

      if aug == 'GAN':
        print(f'Training on GAN - Train Device : {train}  Floorplan : {flp} \n')
        # Replace this with other models to train
        gan_x, gan_y, gan_model = train_gan(train_x, train_y, noise_dim=int(train_x.shape[-1]), batch_size=int(train_x.shape[0]), epochs=20)
        model_params(gan_model, f'{aug}_{train}_{flp}')
        gan_org_x = np.concatenate((train_x, gan_x), axis=0)
        gan_org_y = np.concatenate((train_y, gan_y), axis=0)
        print(f'\n************ Training Model : {aug} {flp} + {model_name} ************\n')

        # [JJ] [IB] : Add more if conditions for other models
        if model_name == 'DNN':
          model = train_DNN(gan_org_x, gan_org_y, 700, batch_size=128)
          save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
          model.save(f'{save_name}.keras')
          model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

        print(f'Training GAN Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')

        if model_name == 'CATBOOST':
          model = train_Catboost(gan_org_x, gan_org_y, 50)
          save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
          model.save_model(f'{save_name}.cbm')
          model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

        print(f'Training GAN Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')

        if model_name == 'XGBOOST':
          model = train_XGboost(gan_org_x, gan_org_y)
          save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
          model.save_model(f'{save_name}.json')
          #model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

        print(f'Training GAN Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')

      if aug == 'FGSM':
        for eps in epsilon:
          print(f'Training on FGSM with Epsilon = {eps} - Train Device : {train}  Floorplan : {flp} \n')
          if model_name == 'XGBOOST':
            model = XGBClassifier().load_model(f'ORIGINAL_{model_name}_eps_0_train_{train}_{flp}.json')
          elif model_name == 'CATBOOST':
            model = CatBoostClassifier().load_model(f'ORIGINAL_{model_name}_eps_0_train_{train}_{flp}.cbm')
          else:
            model = tf.keras.models.load_model(f'ORIGINAL_{model_name}_eps_0_train_{train}_{flp}.keras')
          if model:
            fgsm_x, fgsm_y = train_fgsm(train_x, train_y, eps, model, aug, model_name, train, flp)
            fgsm_org_x = np.concatenate((train_x, fgsm_x), axis=0)
            fgsm_org_y = np.concatenate((train_y, fgsm_y), axis=0)
            print(f'\n************ Training Model : {aug} {eps} {flp} + {model_name} ************\n')

            # [JJ] [IB] : Add more if conditions for other models
            if model_name == 'DNN':
              model = train_DNN(fgsm_org_x, fgsm_org_y, 700, batch_size=128)
              save_name = f'{aug}_{model_name}_eps_{eps}_train_{train}_{flp}'
              model.save(f'{save_name}.keras')
              model_params(model, f'{aug}_{model_name}_{train}_eps_{eps}_{flp}')
            if model_name == 'CATBOOST':
              model = train_Catboost(fgsm_org_x, fgsm_org_y, 50)
              save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
              model.save_model(f'{save_name}.cbm')
              model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')
            if model_name == 'XGBOOST':
              model = train_XGboost(fgsm_org_x, fgsm_org_y)
              save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
              model.save_model(f'{save_name}.json')
              #model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

          else:
            print(f'Pre-trained model required for {aug}')
        print(f'Training FGSM Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')

      if aug == 'PGD':
        for eps in epsilon:
          print(f'Training on PGD with Epsilon = {eps} - Train Device : {train}  Floorplan : {flp} \n')
          if model_name == 'XGBOOST':
            model = XGBClassifier().load_model(f'ORIGINAL_{model_name}_eps_0_train_{train}_{flp}.json')
          elif model_name == 'CATBOOST':
            model = CatBoostClassifier().load_model(f'ORIGINAL_{model_name}_eps_0_train_{train}_{flp}.cbm')
          else:
            model = tf.keras.models.load_model(f'ORIGINAL_{model_name}_eps_0_train_{train}_{flp}.keras')
          if model:
            pgd_x, pgd_y = train_pgd(train_x, train_y, eps, model)
            pgd_org_x = np.concatenate((train_x, pgd_x), axis=0)
            pgd_org_y = np.concatenate((train_y, pgd_y), axis=0)
            print(f'\n************ Training Model : {aug} {eps} {flp} + {model_name} ************\n')

            # [JJ] [IB] : Add more if conditions for other models
            if model_name == 'DNN':
              model = train_DNN(pgd_org_x, pgd_org_y, 700, batch_size=128)
              save_name = f'{aug}_{model_name}_eps_{eps}_train_{train}_{flp}'
              model.save(f'{save_name}.keras')
              model_params(model, f'{aug}_{model_name}_{train}_eps_{eps}_{flp}')
            if model_name == 'CATBOOST':
              model = train_Catboost(pgd_org_x, pgd_org_y, 50)
              save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
              model.save_model(f'{save_name}.cbm')
              model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')
            if model_name == 'XGBOOST':
              model = train_XGboost(pgd_org_x, pgd_org_y)
              save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
              model.save_model(f'{save_name}.json')
              #model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')
          else:
            print(f'Pre-trained model required for {aug}')
        print(f'Training PGD Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')

      if aug == 'MIM':
        for eps in epsilon:
          print(f'Training on MIM with Epsilon = {eps} - Train Device : {train}  Floorplan : {flp} \n')
          if model_name == 'XGBOOST':
            model = XGBClassifier().load_model(f'ORIGINAL_{model_name}_eps_0_train_{train}_{flp}.json')
          elif model_name == 'CATBOOST':
            model = CatBoostClassifier().load_model(f'ORIGINAL_{model_name}_eps_0_train_{train}_{flp}.cbm')
          else:
            model = tf.keras.models.load_model(f'ORIGINAL_{model_name}_eps_0_train_{train}_{flp}.keras')
          if model:
            mim_x, mim_y = train_mim(train_x, train_y, eps, model)
            mim_org_x = np.concatenate((train_x, mim_x), axis=0)
            mim_org_y = np.concatenate((train_y, mim_y), axis=0)
            print(f'\n************ Training Model : {aug} {eps} {flp} + {model_name} ************\n')

            # [JJ] [IB] : Add more if conditions for other models
            if model_name == 'DNN':
              model = train_DNN(mim_org_x, mim_org_y, 700, batch_size=128)
              save_name = f'{aug}_{model_name}_eps_{eps}_train_{train}_{flp}'
              model.save(f'{save_name}.keras')
              model_params(model, f'{aug}_{model_name}_{train}_eps_{eps}_{flp}')

            if model_name == 'CATBOOST':
              model = train_Catboost(mim_org_x, mim_org_y, 50)
              save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
              model.save_model(f'{save_name}.cbm')
              model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

            if model_name == 'XGBOOST':
              model = train_XGboost(mim_org_x, mim_org_y)
              save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
              model.save_model(f'{save_name}.json')
              #model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

            print(f'Training MIM Complete .... \n')
            print('--------------------------------------------------------------------------------------------')
            print('--------------------------------------------------------------------------------------------')

          else:
            print(f'Pre-trained model required for {aug}')
        print(f'Training MIM Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')
      if aug == 'BIM':
        for eps in epsilon:
          print(f'Training on BIM with Epsilon = {eps} - Train Device : {train}  Floorplan : {flp} \n')
          if model_name == 'XGBOOST':
            model = XGBClassifier().load_model(f'ORIGINAL_{model_name}_eps_0_train_{train}_{flp}.json')
          elif model_name == 'CATBOOST':
            model = CatBoostClassifier().load_model(f'ORIGINAL_{model_name}_eps_0_train_{train}_{flp}.cbm')
          else:
            model = tf.keras.models.load_model(f'ORIGINAL_{model_name}_eps_0_train_{train}_{flp}.keras')
          if model:
            bim_x, bim_y = train_bim(train_x, train_y, eps, model)
            bim_org_x = np.concatenate((train_x, bim_x), axis=0)
            bim_org_y = np.concatenate((train_y, bim_y), axis=0)
            print(f'\n************ Training Model : {aug} {eps} {flp} + {model_name} ************\n')

            # [JJ] [IB] : Add more if conditions for other models
            if model_name == 'DNN':
              model = train_DNN(bim_org_x, bim_org_y, 700, batch_size=128)
              save_name = f'{aug}_{model_name}_eps_{eps}_train_{train}_{flp}'
              model.save(f'{save_name}.keras')
              model_params(model, f'{aug}_{model_name}_{train}_eps_{eps}_{flp}')
            if model_name == 'CATBOOST':
              model = train_Catboost(bim_org_x, bim_org_y, 50)
              save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
              model.save_model(f'{save_name}.cbm')
              model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')
            if model_name == 'XGBOOST':
              model = train_XGboost(bim_org_x, bim_org_y)
              save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
              model.save_model(f'{save_name}.json')
              #model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

          else:
            print(f'Pre-trained model required for {aug}')
        print(f'Training BIM Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')

      if aug == 'CW':
        for eps in epsilon:
          print(f'Training on BIM with Epsilon = {eps} - Train Device : {train}  Floorplan : {flp} \n')
          if model_name == 'XGBOOST':
            model = XGBClassifier().load_model(f'ORIGINAL_{model_name}_eps_0_train_{train}_{flp}.json')
          elif model_name == 'CATBOOST':
            model = CatBoostClassifier().load_model(f'ORIGINAL_{model_name}_eps_0_train_{train}_{flp}.cbm')
          else:
            model = tf.keras.models.load_model(f'ORIGINAL_{model_name}_eps_0_train_{train}_{flp}.keras')
          if model:
            cw_x, cw_y = train_cw(train_x, train_y, eps, model)
            cw_org_x = np.concatenate((train_x, cw_x), axis=0)
            cw_org_y = np.concatenate((train_y, cw_y), axis=0)
            print(f'\n************ Training Model : {aug} {eps} {flp} + {model_name} ************\n')

            # [JJ] [IB] : Add more if conditions for other models
            if model_name == 'DNN':
              model = train_DNN(cw_org_x, cw_org_y, 700, batch_size=128)
              save_name = f'{aug}_{model_name}_eps_{eps}_train_{train}_{flp}'
              model.save(f'{save_name}.keras')
              model_params(model, f'{aug}_{model_name}_{train}_eps_{eps}_{flp}')
            if model_name == 'CATBOOST':
              model = train_Catboost(cw_org_x, cw_org_y, 50)
              save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
              model.save_model(f'{save_name}.cbm')
              model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')
            if model_name == 'XGBOOST':
              model = train_XGboost(cw_org_x, cw_org_y)
              save_name = f'{aug}_{model_name}_eps_0_train_{train}_{flp}'
              model.save_model(f'{save_name}.json')
              #model_params(model, f'{aug}_{model_name}_{train}_eps_0_{flp}')

          else:
            print(f'Pre-trained model required for {aug}')
        print(f'Training CW Complete .... \n')
        print('--------------------------------------------------------------------------------------------')
        print('--------------------------------------------------------------------------------------------')


zip_filename = f'{model_name}_models.zip'
# Create a Zip
with ZipFile(zip_filename, 'w') as zipf:
    for foldername, subfolders, filenames in os.walk('.'):
        for filename in filenames:
            if filename.endswith('.keras') or filename.endswith('.txt') or filename.endswith('.cbm') or filename.endswith('.json'):
                filepath = os.path.join(foldername, filename)
                zipf.write(filepath, os.path.relpath(filepath, start='.'))

files.download(zip_filename)


KeyboardInterrupt: 

In [ ]:
#Extra Download cell, for if the above does not finish
zip_filename = f'{model_name}_models.zip'
# Create a Zip
with ZipFile(zip_filename, 'w') as zipf:
    for foldername, subfolders, filenames in os.walk('.'):
        for filename in filenames:
            if filename.endswith('.keras') or filename.endswith('.txt') or filename.endswith('.cbm') or filename.endswith('.json'):
                filepath = os.path.join(foldername, filename)
                zipf.write(filepath, os.path.relpath(filepath, start='.'))

files.download(zip_filename)

In [48]:
def prediction_classes_maker(model, predictions):
  if model_name == 'CATBOOST':
    prediction_classes = []
    for prediction in predictions:
      prediction_classes.append(prediction[0])
    prediction_classes = np.array(prediction_classes)
  else:
    prediction_classes = np.argmax(predictions, axis=1)
  return prediction_classes

In [ ]:
%%capture
model_name = 'DNN'
# Testing the Trained Models
summary_data = []
for att in attack:
  if att == 'ORIGINAL':
    for aug in augmented_data:
      for flp in floorplan:
        print('--------------------------------------------------------------------------------------------')
        print(f'Testing on Original RSS\n')
        print('--------------------------------------------------------------------------------------------')
        if aug in mode:
          train_eps = 0
          test_eps = 0.1
          try:
            if model_name == 'XGBOOST':
              print(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.json')
              model = XGBClassifier()
              model.load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.json')
              print(model)
            elif model_name == 'CATBOOST':
              model = CatBoostClassifier().load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.cbm')
            else:
              model = tf.keras.models.load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.keras')
          except:
            print(f'[WARNING] : {aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.keras Model not found!')

          for device in dev:
            print(f'\n Test dev -> {device}  flp -> {flp} Model -> {aug} {model_name} Attack -> {att} Epsilon -> N/A \n')
            # Initial CI only !!!
            ci_val = 0
            train_aps_value = train_ap_dict[f'{train}_{flp}']
            test_x, test_y = test_data(ci_val, train_aps_value, device, flp)
            predictions = model.predict(test_x)
            prediction_classes = np.argmax(predictions, axis=1)

            mean_error = np.abs(prediction_classes - test_y)
            df = pd.DataFrame({'Prediction': prediction_classes, 'Actual': test_y, 'Error': mean_error})


            summary_data.append({
                'Floorplan': flp,
                'Device': device,
                'Training Data': aug,
                'Training Epsilon': train_eps,
                'Attack': att,
                'Inference Epsilon': test_eps,
                'Mean Error': np.mean(mean_error),
                'Median Error': np.median(mean_error),
                'Min Error': np.min(mean_error),
                'Max Error': np.max(mean_error)
            })
            print(f'Mean Error : {np.mean(mean_error)} Median Error : {np.median(mean_error)}')

            df.to_csv(f'{aug}_{model_name}_train_{train}_{flp}_test_{device}.csv', index=False)

          else:
            for train_eps in epsilon:
              try:
                if model_name == 'XGBOOST':
                  model = XGBClassifier().load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.json')
                elif model_name == 'CATBOOST':
                  model = CatBoostClassifier().load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.cbm')
                else:
                  model = tf.keras.models.load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.keras')
                flag = 1
              except:
                print(f'[WARNING] : {aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.keras Model not found!')
                flag = -1

              if flag == 1:
                for device in dev:
                  print(f'\n Test dev -> {device}  flp -> {flp} Model -> {aug} {model_name} Train Epsilon -> {train_eps} Attack -> {att}\n')
                  # Initial CI only !!!
                  ci_val = 0
                  train_aps_value = train_ap_dict[f'{train}_{flp}']
                  test_x, test_y = test_data(ci_val, train_aps_value, device, flp)
                  fgsm_x, fgsm_y = train_fgsm(test_x, test_y, 0, model)
                  predictions = model.predict(fgsm_x)
                  prediction_classes = np.argmax(predictions, axis=1)

                  mean_error = np.abs(prediction_classes - test_y)
                  df = pd.DataFrame({'Prediction': prediction_classes, 'Actual': test_y, 'Error': mean_error})


                  summary_data.append({
                      'Floorplan': flp,
                      'Device': device,
                      'Training Data': aug,
                      'Training Epsilon': train_eps,
                      'Attack': att,
                      'Inference Epsilon': test_eps,
                      'Mean Error': np.mean(mean_error),
                      'Median Error': np.median(mean_error),
                      'Min Error': np.min(mean_error),
                      'Max Error': np.max(mean_error)
                  })
                  print(f'Mean Error : {np.mean(mean_error)} Median Error : {np.median(mean_error)}')

                  df.to_csv(f'{aug}_{model_name}_train_{train}_trainEPS_{train_eps}_{flp}_test_{device}.csv', index=False)
      # summary.to_csv(f'Summary_{aug}_{model_name}_train_{train}_{flp}_test_{device}.csv', index=False)
    print('--------------------------------------------------------------------------------------------')
    print(f'Inference Complete .... \n')
    print('--------------------------------------------------------------------------------------------')

  if att == 'FGSM':
    for aug in augmented_data:
      for flp in floorplan:
        print('--------------------------------------------------------------------------------------------')
        print(f'Testing on FGSM RSS\n')
        print('--------------------------------------------------------------------------------------------')
        for train_eps in epsilon:
          try:
            if model_name == 'XGBOOST':
              model = XGBClassifier().load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.json')
            elif model_name == 'CATBOOST':
              model = CatBoostClassifier().load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.cbm')
            else:
              model = tf.keras.models.load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.keras')
            flag = 1
          except:
            print(f'[WARNING] : {aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.keras Model not found!')
            flag = -1

          if flag == 1:
            for device in dev:
              for test_eps in epsilon:
                print(f'\n Test dev -> {device}  flp -> {flp} Model -> {aug} {model_name} Train Epsilon -> {train_eps} Attack -> {att} Test Epsilon -> {test_eps} \n')
                # Initial CI only !!!
                ci_val = 0
                train_aps_value = train_ap_dict[f'{train}_{flp}']
                test_x, test_y = test_data(ci_val, train_aps_value, device, flp)
                fgsm_x, fgsm_y = train_fgsm(test_x, test_y, test_eps, model, aug, model_name, train, flp)
                predictions = model.predict(fgsm_x)
                prediction_classes = np.argmax(predictions, axis=1)

                mean_error = np.abs(prediction_classes - test_y)
                df = pd.DataFrame({'Prediction': prediction_classes, 'Actual': test_y, 'Error': mean_error})

                summary_data.append({
                    'Floorplan': flp,
                    'Device': device,
                    'Training Data': aug,
                    'Training Epsilon': train_eps,
                    'Attack': att,
                    'Inference Epsilon': test_eps,
                    'Mean Error': np.mean(mean_error),
                    'Median Error': np.median(mean_error),
                    'Min Error': np.min(mean_error),
                    'Max Error': np.max(mean_error)
                })
                print(f'Mean Error : {np.mean(mean_error)} Median Error : {np.median(mean_error)}')

                df.to_csv(f'{aug}_{model_name}_train_{train}_trainEPS_{train_eps}_{flp}_test_{device}_testEPS_{test_eps}.csv', index=False)
      # summary.to_csv(f'Summary_{aug}_{model_name}_train_{train}_{flp}_test_{device}.csv', index=False)
    print('--------------------------------------------------------------------------------------------')
    print(f'Inference Complete .... \n')
    print('--------------------------------------------------------------------------------------------')

  if att == 'PGD':
    for aug in augmented_data:
      for flp in floorplan:
        print('--------------------------------------------------------------------------------------------')
        print(f'Testing on PGD RSS\n')
        print('--------------------------------------------------------------------------------------------')
        for train_eps in epsilon:
          try:
            if model_name == 'XGBOOST':
              model = XGBClassifier().load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.json')
            elif model_name == 'CATBOOST':
              model = CatBoostClassifier().load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.cbm')
            else:
              model = tf.keras.models.load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.keras')
            flag = 1
          except:
            print(f'[WARNING] : {aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.keras Model not found!')
            flag = -1

          if flag == 1:
            for device in dev:
              for test_eps in epsilon:
                print(f'\n Test dev -> {device}  flp -> {flp} Model -> {aug} {model_name} Train Epsilon -> {train_eps} Attack -> {att} Test Epsilon -> {test_eps} \n')
                ci_val = 0
                train_aps_value = train_ap_dict[f'{train}_{flp}']
                test_x, test_y = test_data(ci_val, train_aps_value, device, flp)
                pgd_x, pgd_y = train_pgd(test_x, test_y, test_eps, model)
                predictions = model.predict(pgd_x)
                prediction_classes = np.argmax(predictions, axis=1)

                mean_error = np.abs(prediction_classes - test_y)
                df = pd.DataFrame({'Prediction': prediction_classes, 'Actual': test_y, 'Error': mean_error})

                summary_data.append({
                    'Floorplan': flp,
                    'Device': device,
                    'Training Data': aug,
                    'Training Epsilon': train_eps,
                    'Attack': att,
                    'Inference Epsilon': test_eps,
                    'Mean Error': np.mean(mean_error),
                    'Median Error': np.median(mean_error),
                    'Min Error': np.min(mean_error),
                    'Max Error': np.max(mean_error)
                })
                print(f'Mean Error : {np.mean(mean_error)} Median Error : {np.median(mean_error)}')

                df.to_csv(f'{aug}_{model_name}_train_{train}_trainEPS_{train_eps}_{flp}_test_{device}_testEPS_{test_eps}.csv', index=False)
      # summary.to_csv(f'Summary_{aug}_{model_name}_train_{train}_{flp}_test_{device}.csv', index=False)
    print('--------------------------------------------------------------------------------------------')
    print(f'Inference Complete .... \n')
    print('--------------------------------------------------------------------------------------------')

  if att == 'MIM':
      for aug in augmented_data:
        for flp in floorplan:
          print('--------------------------------------------------------------------------------------------')
          print(f'Testing on MIM RSS\n')
          print('--------------------------------------------------------------------------------------------')
          for train_eps in epsilon:
            try:
              if model_name == 'XGBOOST':
                model = XGBClassifier().load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.json')
              elif model_name == 'CATBOOST':
                model = CatBoostClassifier().load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.cbm')
              else:
                model = tf.keras.models.load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.keras')
              flag = 1
            except:
              print(f'[WARNING] : {aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.keras Model not found!')
              flag = -1

            if flag == 1:
              for device in dev:
                for test_eps in epsilon:
                  print(f'\n Test dev -> {device}  flp -> {flp} Model -> {aug} {model_name} Train Epsilon -> {train_eps} Attack -> {att} Test Epsilon -> {test_eps} \n')
                  # Initial CI only !!!
                  ci_val = 0
                  train_aps_value = train_ap_dict[f'{train}_{flp}']
                  test_x, test_y = test_data(ci_val, train_aps_value, device, flp)
                  mim_x, mim_y = train_mim(test_x, test_y, test_eps, model)
                  predictions = model.predict(mim_x)
                  prediction_classes = np.argmax(predictions, axis=1)

                  mean_error = np.abs(prediction_classes - test_y)
                  df = pd.DataFrame({'Prediction': prediction_classes, 'Actual': test_y, 'Error': mean_error})

                  summary_data.append({
                      'Floorplan': flp,
                      'Device': device,
                      'Training Data': aug,
                      'Training Epsilon': train_eps,
                      'Attack': att,
                      'Inference Epsilon': test_eps,
                      'Mean Error': np.mean(mean_error),
                      'Median Error': np.median(mean_error),
                      'Min Error': np.min(mean_error),
                      'Max Error': np.max(mean_error)
                  })
                  print(f'Mean Error : {np.mean(mean_error)} Median Error : {np.median(mean_error)}')

                  df.to_csv(f'{aug}_{model_name}_train_{train}_trainEPS_{train_eps}_{flp}_test_{device}_testEPS_{test_eps}.csv', index=False)
        # summary.to_csv(f'Summary_{aug}_{model_name}_train_{train}_{flp}_test_{device}.csv', index=False)
      print('--------------------------------------------------------------------------------------------')
      print(f'Inference Complete .... \n')
      print('--------------------------------------------------------------------------------------------')

  if att == 'BIM':
      for aug in augmented_data:
        for flp in floorplan:
          print('--------------------------------------------------------------------------------------------')
          print(f'Testing on MIM RSS\n')
          print('--------------------------------------------------------------------------------------------')
          for train_eps in epsilon:
            try:
              if model_name == 'XGBOOST':
                model = XGBClassifier().load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.json')
              elif model_name == 'CATBOOST':
                model = CatBoostClassifier().load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.cbm')
              else:
                model = tf.keras.models.load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.keras')
              flag = 1
            except:
              print(f'[WARNING] : {aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.keras Model not found!')
              flag = -1

            if flag == 1:
              for device in dev:
                for test_eps in epsilon:
                  print(f'\n Test dev -> {device}  flp -> {flp} Model -> {aug} {model_name} Train Epsilon -> {train_eps} Attack -> {att} Test Epsilon -> {test_eps} \n')
                  # Initial CI only !!!
                  ci_val = 0
                  train_aps_value = train_ap_dict[f'{train}_{flp}']
                  test_x, test_y = test_data(ci_val, train_aps_value, device, flp)
                  bim_x, bim_y = train_bim(test_x, test_y, test_eps, model)
                  predictions = model.predict(bim_x)
                  prediction_classes = np.argmax(predictions, axis=1)

                  mean_error = np.abs(prediction_classes - test_y)
                  df = pd.DataFrame({'Prediction': prediction_classes, 'Actual': test_y, 'Error': mean_error})

                  summary_data.append({
                      'Floorplan': flp,
                      'Device': device,
                      'Training Data': aug,
                      'Training Epsilon': train_eps,
                      'Attack': att,
                      'Inference Epsilon': test_eps,
                      'Mean Error': np.mean(mean_error),
                      'Median Error': np.median(mean_error),
                      'Min Error': np.min(mean_error),
                      'Max Error': np.max(mean_error)
                  })
                  print(f'Mean Error : {np.mean(mean_error)} Median Error : {np.median(mean_error)}')

                  df.to_csv(f'{aug}_{model_name}_train_{train}_trainEPS_{train_eps}_{flp}_test_{device}_testEPS_{test_eps}.csv', index=False)
        # summary.to_csv(f'Summary_{aug}_{model_name}_train_{train}_{flp}_test_{device}.csv', index=False)
      print('--------------------------------------------------------------------------------------------')
      print(f'Inference Complete .... \n')
      print('--------------------------------------------------------------------------------------------')

  if att == 'CW':
      for aug in augmented_data:
        for flp in floorplan:
          print('--------------------------------------------------------------------------------------------')
          print(f'Testing on MIM RSS\n')
          print('--------------------------------------------------------------------------------------------')
          for train_eps in epsilon:
            try:
              if model_name == 'XGBOOST':
                model = XGBClassifier().load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.json')
              elif model_name == 'CATBOOST':
                model = CatBoostClassifier().load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.cbm')
              else:
                model = tf.keras.models.load_model(f'{aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.keras')
              flag = 1
            except:
              print(f'[WARNING] : {aug}_{model_name}_eps_{train_eps}_train_{train}_{flp}.keras Model not found!')
              flag = -1

            if flag == 1:
              for device in dev:
                for test_eps in epsilon:
                  print(f'\n Test dev -> {device}  flp -> {flp} Model -> {aug} {model_name} Train Epsilon -> {train_eps} Attack -> {att} Test Epsilon -> {test_eps} \n')
                  # Initial CI only !!!
                  ci_val = 0
                  train_aps_value = train_ap_dict[f'{train}_{flp}']
                  test_x, test_y = test_data(ci_val, train_aps_value, device, flp)
                  cw_x, cw_y = train_cw(test_x, test_y, test_eps, model)
                  predictions = model.predict(cw_x)
                  prediction_classes = np.argmax(predictions, axis=1)

                  mean_error = np.abs(prediction_classes - test_y)
                  df = pd.DataFrame({'Prediction': prediction_classes, 'Actual': test_y, 'Error': mean_error})

                  summary_data.append({
                      'Floorplan': flp,
                      'Device': device,
                      'Training Data': aug,
                      'Training Epsilon': train_eps,
                      'Attack': att,
                      'Inference Epsilon': test_eps,
                      'Mean Error': np.mean(mean_error),
                      'Median Error': np.median(mean_error),
                      'Min Error': np.min(mean_error),
                      'Max Error': np.max(mean_error)
                  })
                  print(f'Mean Error : {np.mean(mean_error)} Median Error : {np.median(mean_error)}')

                  df.to_csv(f'{aug}_{model_name}_train_{train}_trainEPS_{train_eps}_{flp}_test_{device}_testEPS_{test_eps}.csv', index=False)
        # summary.to_csv(f'Summary_{aug}_{model_name}_train_{train}_{flp}_test_{device}.csv', index=False)
      print('--------------------------------------------------------------------------------------------')
      print(f'Inference Complete .... \n')
      print('--------------------------------------------------------------------------------------------')

summary = pd.DataFrame(summary_data)
summary.to_csv(f'Summary_{model_name}.csv', index=False)
zip_filename = f'{model_name}_complete.zip'
# Create a Zip
with ZipFile(zip_filename, 'w') as zipf:
    for foldername, subfolders, filenames in os.walk('.'):
        for filename in filenames:
            if filename.endswith('.keras') or filename.endswith('.txt') or filename.endswith('.csv'):
                filepath = os.path.join(foldername, filename)
                zipf.write(filepath, os.path.relpath(filepath, start='.'))

files.download(zip_filename)


In [ ]:
''' Author - Danish Gufran
           - Jadon Johnson
           - Isaac Blair
           - Sudeep Pasricha

EPIC Lab : Colorado State University, Fort Collins

<Danish.Gufran@colostate.edu>
<Jadon.Johnson@colostate.edu>
<Isaac.Blair@colostate.edu>
<Sudeep@colostate.edu>

Title - Data Augmentation Methods for Wi-Fi RSS Fingerprinting Based Indoor Localization.

Citation :

'''

In [ ]:
summary = pd.DataFrame(summary_data)
summary.to_csv(f'Summary_{model_name}.csv', index=False)
zip_filename = f'{model_name}_complete.zip'
# Create a Zip
with ZipFile(zip_filename, 'w') as zipf:
    for foldername, subfolders, filenames in os.walk('.'):
        for filename in filenames:
            if filename.endswith('.keras') or filename.endswith('.txt') or filename.endswith('.csv'):
                filepath = os.path.join(foldername, filename)
                zipf.write(filepath, os.path.relpath(filepath, start='.'))

files.download(zip_filename)

In [ ]:
#!mkdir dnn_stuff
#!mv DNN_complete.zip dnn_stuff
!mv Summary_DNN_complete.csv dnn_stuff
!mv DNN_models.zip dnn_stuff